In [ ]:
%matplotlib inline
import os, sys
import time
from datetime import datetime
sys.path.append("../model/")
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2

import keras.backend as K
import keras
import tensorflow as tf

from keras.utils import plot_model, generic_utils
from keras_tqdm import TQDMNotebookCallback
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam, SGD

from unet import UNET
from vgan import VGAN, DISCRIMINATOR, PIC_DISCRIMINATOR

from train_utils import extract_contour, fill_mask, to_uint8
from train_utils import PlotCheckpoint, send_report_to_slack, set_directory, mean_iou, plot_sample_image, plot_bg_removal_sample_image
from train_utils import get_disc_batch, np_mean_iou, to_uint8
from data_utils import load_dataset, HumanSegGenerator
from data_utils import clahe_func, gray_func
from data_utils import rotation_func, rescaling_func, flip_func, random_crop_func, random_noise_func, gamma_func, color_gamma_func
from keras.models import model_from_json

In [ ]:
def np_mean_iou(y_pred, y_true, thr=0.5):
    y_pred = (y_pred>thr).astype(np.int)
    y_true = (y_true>thr).astype(np.int)

    intersect = y_pred * y_true
    union = np.ones_like(y_pred) - ((1-y_pred) * (1-y_true))
    return np.mean(np.sum(np.sum(intersect,axis=1),axis=1) / \
                   np.sum(np.sum(union,axis=1),axis=1))

def to_mask(image_batch,thr=-.99):
    images = []
    for image in image_batch:
        mask = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        mask = (mask > thr).astype(np.float32)
        images.append(mask)
    return np.stack(images, axis=0)

In [ ]:
model_name = "gan"
dataset_name = 'train_refined'
h5_path = "../data/baidu-segmentation.h5"

# train setting
batch_size = 1
if batch_size == 1:
    instance = True
else:
    instance = False

nb_epoch = 200

# Model Setting
img_dim = (256,256,3)
output_dim = (256,256,1)
activation = 'LeakyReLU'
bn = True
bg_removal = False
sigmoid = False

if sigmoid:
    # 범위가 0~1이므로 threshold는 0.5
    thr = 0.5
else:
    # 범위가 -1~1이므로 threshold는 0.
    thr = 0.

# Generator Setting
nb_filter = 16
depth = 4
lr = 0.0002
beta_1 = 0.5
loss_function = "mae"
#loss_function = 'binary_crossentropy'
drop_rate = 0.

# Discriminator Setting
nb_filter_disc = 32
lr_disc = 0.0001
beta_1_disc = 0.5
depth_disc = 6
loss_function_disc = "binary_crossentropy"
opt_type = 'adam' # opt_type = 'sgd'
drop_rate = 0.

# gan Setting
alpha = 10

# training Setting
label_smoothing = False
label_flipping = 0.0
round_size = 1000


load = False
load_gen_path = "../results/gan/0706_06/model-arch/generator.json"
load_gen_weight_path = "../results/gan/0706_06/weights/0.883_gen.h5"
load_disc_path = "../results/gan/0706_06/model-arch/discriminator.json"
load_disc_weight_path = "../results/gan/0706_06/weights/0.883_disc.h5"

title = "6th trial - Change Loss function"
description = '''
    generator output은 sigmoid -> tanh으로!
'''

Todo-List

    1. Dropout을 추가하였을 때, 어떤 식으로 동작하는지 알아보기. 
    2. Discriminator의 학습 바꾸었을 때 어떤식으로 동작하는 지 확인
    3. 


In [ ]:
if load:
    with open(load_disc_path) as f:
        discriminator = model_from_json(f.readline())
        discriminator.load_weights(load_disc_weight_path)
    with open(load_gen_path) as f:
        generator = model_from_json(f.readline())
        generator.load_weights(load_gen_weight_path)
    ## gan
    gan = VGAN(generator,discriminator)
else:    
    # create the model
    ## generator
    generator = UNET(img_dim=img_dim,
                depth=depth,
                nb_filter=nb_filter,
                output_dim=output_dim,
                bn=bn,
                instance=instance,
                activation=activation,
                sigmoid=sigmoid)

    ## discriminator
    discriminator = PIC_DISCRIMINATOR(img_dim=img_dim,
                        nb_filter=nb_filter_disc,
                        depth=depth_disc,
                        activation=activation,
                        bn=bn,
                        instance=instance,
                        output_dim=output_dim)
    ## gan
    gan = VGAN(generator,discriminator)

In [ ]:
## set the optimizer
opt_generator = Adam(lr=lr, beta_1=beta_1, beta_2=0.999)
if opt_type == 'sgd':
    opt_discriminator = SGD(lr=lr_disc, momentum=0.9, nesterov=True)
else:
    opt_discriminator = Adam(lr=lr_disc, beta_1=beta_1, beta_2=0.999)

## set the loss function
discriminator.trainable = False
generator.compile(loss=loss_function, optimizer=opt_generator) 

loss = [loss_function, loss_function_disc]
loss_weights = [alpha, 1]
gan.compile(loss=loss_function, loss_weights=loss_weights, optimizer=opt_generator)

discriminator.trainable = True
discriminator.compile(loss=loss_function_disc, optimizer=opt_discriminator)

# create data generator
trainset = load_dataset(dataset_name=dataset_name,h5_path=h5_path)

epoch_size = len(trainset)
train_steps = epoch_size // batch_size

traingen = HumanSegGenerator(trainset,img_dim,
                             batch_size=batch_size,
                             sigmoid=sigmoid,
                             bg_removal=bg_removal,
                             aug_funcs=[rotation_func(), 
                                        rescaling_func(),
                                        flip_func(), 
                                        random_crop_func(),
                                        random_noise_func(),
                                        gamma_func(),
                                        color_gamma_func()],
                             prep_funcs=[])

testset = load_dataset(dataset_name='test',h5_path=h5_path)
test_images, test_profiles = next(HumanSegGenerator(testset,img_dim,
                                                    batch_size=len(testset),
                                                    sigmoid=sigmoid,
                                                    bg_removal=bg_removal,
                                                    prep_funcs=[]))

loss_df =  pd.DataFrame()

In [ ]:
# set the model directory to save 
weights_dir, model_dir, sample_dir = set_directory("../results",model_name)

# draw the architecture diagram of model
model_path = os.path.join(model_dir, "generator.png")
plot_model(generator, to_file=model_path, show_shapes=True)

model_path = os.path.join(model_dir, "discriminator.png")
plot_model(discriminator, to_file=model_path, show_shapes=True)

# serialize model to JSON
model_json = generator.to_json()
with open(os.path.join(model_dir,"generator.json"), "w") as json_file:
    json_file.write(model_json)
    
model_json = discriminator.to_json()
with open(os.path.join(model_dir,"discriminator.json"), "w") as json_file:
    json_file.write(model_json)

# train the Model
best_iou = 0
for e in range(nb_epoch):
    progbar = generic_utils.Progbar(epoch_size)
    disc_losses = []; g_tot_losses = []
    g_losses = []; g_log_losses = []
    
    for _ in range(train_steps//(round_size*2)):
        # Update the discriminator
        # Unfreeze the discriminator
        discriminator.trainable = True
        for i in range(round_size):
            image_batch, profile_batch = next(traingen)
            X_disc, y_disc = get_disc_batch(image_batch, profile_batch, 
                                            generator, i,
                                            label_smoothing=label_smoothing,
                                            label_flipping=label_flipping)
            disc_loss = discriminator.train_on_batch(X_disc, y_disc)
            disc_losses.append(disc_loss);
            
            progbar.add(batch_size, values=[("D logloss", disc_loss)])
            
        
        # update the Generator
        # Freeze the discriminator
        discriminator.trainable = False
        for i in range(round_size):
            image_batch, profile_batch = next(traingen)

            y_gen = np.zeros((image_batch.shape[0], 2))
            y_gen[:, 1] = 1.
            gen_loss = gan.train_on_batch(image_batch, [profile_batch, y_gen])
            
            g_tot_losses.append(gen_loss[0])
            g_losses.append(gen_loss[1])
            g_log_losses.append(gen_loss[2])

            progbar.add(batch_size,values=[("G tot", gen_loss[0]),
                                   ("G {}".format(loss_function), gen_loss[1]),
                                    ("G logloss", gen_loss[2])])
            
        # save the training progress
        gen_batch = generator.predict_on_batch(image_batch)
        iou = np_mean_iou(gen_batch, profile_batch, thr)
        gen_test_batch = generator.predict_on_batch(test_images)
        test_iou = np_mean_iou(gen_test_batch, test_profiles, thr)
        
        loss_df = loss_df.append({
                "epoch" : e,
                "train D logloss" : np.mean(disc_losses),
                "train G tot" : np.mean(g_tot_losses),
                "train G {}".format(loss_function) : np.mean(g_losses),
                "train G logloss": np.mean(g_log_losses),
                "train IOU": iou, "test IOU": test_iou
            },ignore_index=True)
                
        if test_iou > best_iou:
            generator.save_weights(os.path.join(weights_dir,"{:1.3f}_gen.h5".format(best_iou)))
            discriminator.save_weights(os.path.join(weights_dir,"{:1.3f}_disc.h5".format(best_iou)))
            gan.save_weights(os.path.join(weights_dir,"{:1.3f}_gan.h5".format(best_iou)))
            best_iou = test_iou

        disc_losses = []; g_tot_losses = []
        g_losses = []; g_log_losses = []
        print("\n{:03d}_epoch test IOU : {:.4f}".format(e, test_iou))
    
    # Draw the Result
    filename = "{:03d}_epoch_sample.png".format(e)
    sample_path = os.path.join(sample_dir,filename)            
    gen_profiles = generator.predict_on_batch(test_images)
    if bg_removal:
        plot_bg_removal_sample_image(test_images, gen_profiles, sample_path)
    else:
        plot_sample_image(test_images, gen_profiles, sample_path)
        
    # serialize weights to HDF5
    best_idx = loss_df[['test IOU','epoch']].sort_values('test IOU',ascending=False).iloc[0]['epoch']
    best_iou = loss_df['test IOU'].sort_values(ascending=False).iloc[0]
    generator.save_weights(os.path.join(weights_dir,"{:1.3f}_gen.h5".format(best_iou)))
    discriminator.save_weights(os.path.join(weights_dir,"{:1.3f}_disc.h5".format(best_iou)))
    gan.save_weights(os.path.join(weights_dir,"{:1.3f}_gan.h5".format(best_iou)))
    
    # Draw the Training Learning Curve
    loss_df = loss_df.round(4)
    loss_df.to_csv(os.path.join(model_dir, "training.csv"),index=False)
    loss_df[['train IOU','test IOU']].plot().figure.savefig(os.path.join(model_dir,"iou_training.png"))
    loss_df[['train D logloss','train G tot']].plot().figure.savefig(os.path.join(model_dir,"loss_training.png"))
    loss_df[['train G logloss','train G {}'.format(loss_function)]].plot().figure.savefig(os.path.join(model_dir,"gen_loss_training.png"))
    
description = description \
+ "\nmodel_path : {}".format(weights_dir[:-7])\
+ "\nbest_mean_iou : {}".format(best_iou)
                              
image_paths = {
    'gen_model' : os.path.join(model_dir,"generator.png"),
    "disc_model" : os.path.join(model_dir,"discriminator.png"),
    'iou-training' : os.path.join(model_dir,'iou_training.png'),
    'loss-training' : os.path.join(model_dir,'loss_training.png'), 
    'train-process-csv' : os.path.join(model_dir,"training.csv"),
    'test image' : os.path.join(sample_dir,"{:03d}_epoch_sample.png".format(int(best_idx)))
}

send_report_to_slack(title,description,image_paths)